In [ ]:
import click
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from glob import glob
import pandas as pd
import os
import shutil
import joblib
import json
from shutil import copytree
from termcolor import colored


In [ ]:
def main(path):
    splitsort = []
    smaller = []
    
    for i in sorted(glob(os.path.join(path, '*log.csv'))):
        
        splitname = os.path.basename(i).split('_')
        if len(splitname)==3 and int(splitname[0])!=0:
            splitsort.append((int(splitname[0]),i))
        elif len(splitname)==2:
            smaller.append(i)
    
    if len(splitsort)!=0:
        dirs = list(np.array(sorted(splitsort))[:,1]) + smaller
    else:
        dirs = smaller
    loss = []
    for i in dirs:
        # print(f"Processing file: {i}")
        # print(i)
        loss.append(split(i))  # Get only the loss values (assumed to be in the second column)

    
    # plt.plot(*np.vstack(loss).T)
    # plt.xlabel("Iteration")
    # plt.ylabel("Loss")
    # plt.title("Loss Curve")
    # plt.show()
    
    return np.max(np.vstack(loss)[:,0]).astype(int)

def split(path):
    """Helper function to read CSV file and extract required columns."""
    with open(path, 'r') as f:
        save = pd.read_csv(path)[['Iteration','Cost']].dropna()
    return save.values    

In [ ]:
fiveog = [os.path.dirname(i) for i in glob('5qubit/*/*.json') if 'results' not in i]
fivenew = [os.path.dirname(i) for i in glob('5/*/*.json') if 'results' not in i]

In [ ]:
len(fivenew)

In [ ]:
ran_all = []

for i in fiveog:
    newpath = os.path.join('5',os.path.basename(i))
    oldpath = os.path.join('5qubitbackup',os.path.basename(i))
    iters = main(oldpath)

    if iters==999:
        print(oldpath,iters)    
        ran_all.append((os.path.basename(oldpath),iters))
    else:
        try:
            newiters = main(newpath)
            print(colored(newpath, 'green'),colored(newiters, 'blue'))
            print(colored(oldpath, 'red'),colored(iters, 'blue'))
            ran_all.append((os.path.basename(newpath),newiters))
        except:
            print(colored(oldpath, 'yellow'),colored(iters, 'blue'))
    print()

In [ ]:
df = pd.DataFrame(ran_all)

In [ ]:
allname = [os.path.basename(i) for i in fiveog]

In [ ]:
os.getcwd()

In [ ]:
for i in list(set(allname)-set(df[0])):
    iters = main(os.path.join('5qubitbackup',i))
    print(i,iters)
    copytree(os.path.join('5qubitbackup',i),os.path.join('5rerun',i))